In [1]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.0.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.0.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul
import torch_geometric
from torch_geometric.data import DataLoader
from torch_geometric.data import HeteroData
from torch_geometric.utils import negative_sampling
from torch_geometric.transforms import ToUndirected
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GATConv, to_hetero
from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 515.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 98.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 92.1 MB/s eta 0:00:00:00:01
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 44.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 93.4 MB/s eta 0:00:00:00

In [2]:
data_path = '/kaggle/input/mooccubex/train_df.csv' # Data path
df = pd.read_csv(data_path)

# Convert course interact string from feature column to list
def str_to_list(x): # Function to split user's course interact list from string.
    return x[1:-1].split(',')

df['feature'] = df['feature'].apply(str_to_list) # Apply str_to_list to course interact string.
df['feature'] = df['feature'].apply(lambda x: [int(i) for i in x]) # Convert every course type to int.

# Drop column feature_time.
df = df.drop(columns=['feature_time'])

# Explode df with feature column.
exploded_df = df.explode('feature')
exploded_df.reset_index(drop=True)
exploded_df['feature'] = exploded_df['feature'].astype('int64')

# Extract columns
user_col = torch.tensor(exploded_df['user'].values, dtype=torch.int64)
course_col = torch.tensor(exploded_df['feature'].values, dtype=torch.int64)

train_edge_index = torch.stack([user_col, course_col], dim=0)

# Update num_users and num_courses
num_users = user_col.max().item() + 1
num_courses = course_col.max().item() + 1

# Shift course indices and create SparseTensor
train_sparse_edge_index = SparseTensor(
    row = user_col,
    col = course_col + num_users,
    sparse_sizes=(num_users + num_courses, num_users + num_courses)
)

print(train_sparse_edge_index)

data_path = '/kaggle/input/mooccubex/val_df.csv'
df = pd.read_csv(data_path)
df = df[df['val_label'] < num_courses]

test_user_col = torch.tensor(df['user'].values, dtype=torch.int64)
test_course_col = torch.tensor(df['val_label'].values, dtype=torch.int64)

test_edge_index = torch.stack([test_user_col, test_course_col], dim=0)

test_course_col_shifted = test_course_col + num_users
test_sparse_edge_index = SparseTensor(
    row=test_user_col,
    col=test_course_col_shifted,
    sparse_sizes=(num_users + num_courses, num_users + num_courses)
)

print(test_sparse_edge_index)

# Load course - school data: Course - school interact
data_path = '/kaggle/input/mooccubex/mapped_course-school.csv' # Data path
df = pd.read_csv(data_path) # Read data
df = df[df['school'].isin(exploded_df['feature'].unique())] # Filter valid course exists in course - school relastionships.
school_mapping = {school: idx for idx, school in enumerate(df['school'].unique())}
df['school'] = df['school'].map(school_mapping)

# Prepare course - school edge index
course_col = torch.tensor(df['course'].values, dtype=torch.int64)
school_col = torch.tensor(df['school'].values, dtype=torch.int64)
course_school_edge_index = torch.stack([course_col, school_col], dim=0) # Create course - school edge index
num_schools = len(school_mapping)

course_school_sparse_edge_index = SparseTensor(
    row = school_col + num_users + num_courses,
    col = course_col + num_users,
    sparse_sizes=(num_users + num_courses + num_schools, num_users + num_courses + num_schools)
)

print(course_school_sparse_edge_index)

# Load course - teacher data: Course - teacher interact
data_path = '/kaggle/input/mooccubex/mapped_course-teacher.csv' # Data path
df = pd.read_csv(data_path) # Read data
df = df[df['id'].isin(exploded_df['feature'].unique())] # Filter valid course exists in course - teacher relastionships.
teacher_mapping = {teacher: idx for idx, teacher in enumerate(df['teachers'].unique())}
df['teachers'] = df['teachers'].map(teacher_mapping)

# Prepare course - teacher edge index
course_col = torch.tensor(df['id'].values, dtype=torch.int64)
teacher_col = torch.tensor(df['teachers'].values, dtype=torch.int64)
course_teacher_edge_index = torch.stack([course_col, teacher_col], dim=0) # Create course - teacher edge index
num_teachers = len(teacher_mapping)

course_teacher_sparse_edge_index = SparseTensor(
    row = teacher_col + num_users + num_courses + num_schools,
    col = course_col + num_users,
    sparse_sizes=(num_users + num_courses + num_schools + num_teachers, num_users + num_courses + num_schools + num_teachers)
)

print(course_teacher_sparse_edge_index)

# Load course - field data: Course - field interact
data_path = '/kaggle/input/mooccubex/mapped_course-field.csv' # Data path
df = pd.read_csv(data_path) # Read data
df = df[df['id'].isin(exploded_df['feature'].unique())] # Filter valid course exists in course - field relastionships.
field_mapping = {field: idx for idx, field in enumerate(df['field'].unique())}
df['field'] = df['field'].map(field_mapping)

# Prepare course - field edge index
course_col = torch.tensor(df['id'].values, dtype=torch.int64)
field_col = torch.tensor(df['field'].values, dtype=torch.int64)
course_field_edge_index = torch.stack([course_col, field_col], dim=0) # Create course - field edge index
num_fields = len(field_mapping)

course_field_sparse_edge_index = SparseTensor(
    col = field_col + num_users + num_courses + num_schools + num_teachers,
    row = course_col + num_users,
    sparse_sizes=(num_users + num_courses + num_schools + num_teachers + num_fields, num_users + num_courses + num_schools + num_teachers + num_fields)
)

print(course_field_sparse_edge_index)

SparseTensor(row=tensor([    0,     0,     0,  ..., 99969, 99969, 99969]),
             col=tensor([ 99970,  99971,  99972,  ..., 100235, 100410, 101954]),
             size=(102797, 102797), nnz=1796450, density=0.02%)
SparseTensor(row=tensor([    0,     1,     2,  ..., 99967, 99968, 99969]),
             col=tensor([ 99973,  99971,  99975,  ..., 102158, 102045, 101687]),
             size=(102797, 102797), nnz=99970, density=0.00%)
SparseTensor(row=tensor([102797, 102797, 102797,  ..., 103218, 103219, 103220]),
             col=tensor([ 99971,  99972,  99973,  ..., 101245, 101256, 101696]),
             size=(103221, 103221), nnz=2850, density=0.00%)
SparseTensor(row=tensor([103221, 103221, 103221,  ..., 112171, 112172, 112173]),
             col=tensor([100216, 100318, 100603,  ..., 102608, 102608, 101696]),
             size=(112174, 112174), nnz=10651, density=0.00%)
SparseTensor(row=tensor([ 99972,  99973,  99979,  99979,  99981,  99982,  99982,  99984,  99986,
                  

In [3]:
data_path = '/kaggle/input/kg-final/kg_final.txt' # Data path
df = pd.read_csv(data_path, sep=" ", header=None, names=['h', 'r', 't'])
df = df[df['h'].isin(exploded_df['feature'].unique())] # Filter valid course exists in course - field relastionships.
other_mapping = {other: idx for idx, other in enumerate(df['t'].unique())}
df['t'] = df['t'].map(other_mapping)

# Prepare course - field edge index
course_col = torch.tensor(df['h'].values, dtype=torch.int64)
other_col = torch.tensor(df['t'].values, dtype=torch.int64)
other_edge_type = torch.tensor(df['r'].values, dtype=torch.int64) + 2
course_other_edge_index = torch.stack([course_col, other_col], dim=0) # Create course - field edge index
num_others = len(other_mapping)

course_other_sparse_edge_index = SparseTensor(
    col = other_col + num_users + num_courses,
    row = course_col + num_users,
    sparse_sizes=(num_users + num_courses + num_others, num_users + num_courses + num_others)
)

print(course_other_sparse_edge_index)

SparseTensor(row=tensor([ 99970,  99970,  99971,  ..., 102795, 102795, 102796]),
             col=tensor([103207, 106885, 102821,  ..., 103236, 107473, 103124]),
             size=(110265, 110265), nnz=68772, density=0.00%)


In [4]:
concat_row = torch.cat([train_sparse_edge_index.storage.row(), train_sparse_edge_index.storage.col(), course_other_sparse_edge_index.storage.row()], dim=0)
concat_col = torch.cat([train_sparse_edge_index.storage.col(), train_sparse_edge_index.storage.row(), course_other_sparse_edge_index.storage.col()], dim=0)

edge_type = torch.cat([torch.zeros_like(train_sparse_edge_index.storage.row()), torch.ones_like(train_sparse_edge_index.storage.col()), other_edge_type], dim=0)

num_nodes = max(concat_row.max().item(), concat_col.max().item()) + 1
concatenated_sparse_edge_index = SparseTensor(
    row=concat_row,
    col=concat_col,
    sparse_sizes=(num_nodes, num_nodes)
)

print(concatenated_sparse_edge_index)

SparseTensor(row=tensor([     0,      0,      0,  ..., 102795, 102796, 102796]),
             col=tensor([ 99970,  99971,  99972,  ..., 107473,  96170, 103124]),
             size=(110265, 110265), nnz=3661672, density=0.03%)


In [5]:
import torch
from torch import nn
from torch_geometric.nn import GATv2Conv

class HeteroGATModel(nn.Module):
    def __init__(self, in_dim = 64, hidden_dim = 32, out_dim = 16, num_layers = 3, heads=2, dropout=0.3):
        super(HeteroGATModel, self).__init__()

        # Define embeddings for each type of node
        self.user_entities_embeddings = nn.Embedding(num_users + num_courses + num_others, in_dim)
        self.gat_layers = nn.ModuleList()

        self.gat_layers.append(GATv2Conv(in_dim, hidden_dim, heads=heads, dropout=dropout, add_self_loops=False))
        for _ in range(num_layers - 2):
            self.gat_layers.append(GATv2Conv(hidden_dim * heads, hidden_dim, heads=heads, dropout=dropout, add_self_loops=False))
        self.gat_layers.append(GATv2Conv(hidden_dim * heads, out_dim, heads=1, concat=False, dropout=dropout, add_self_loops=False))

        self.reset_parameters()

    def reset_parameters(self):
        # Initialize embeddings
        nn.init.xavier_uniform_(self.user_entities_embeddings.weight)

        for gat_layer in self.gat_layers:
            gat_layer.reset_parameters()

    def forward(self, edge_index):
        x_initial = self.user_entities_embeddings.weight
        x = x_initial.clone()  # Clone để giữ nguyên embedding ban đầu

        for gat_layer in self.gat_layers[:-1]:
            x = F.elu(gat_layer(x, edge_index))

        x = self.gat_layers[-1](x, edge_index)

        user_emb_final, course_emb_final, other_emb_final = torch.split(x, [num_users, num_courses, num_others], dim=0)
        user_emb_initial, course_emb_initial, _  = torch.split(x_initial, [num_users, num_courses, num_others], dim=0)
        
        return user_emb_final, user_emb_initial, course_emb_final, course_emb_initial, other_emb_final

In [8]:
def RecallPrecision_at_K(groundTruth, r, k):
    num_correct_pred = torch.sum(r, dim=-1)
    user_num_liked = torch.Tensor([len(groundTruth[i]) for i in range(len(groundTruth))])
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    
    return recall.item(), precision.item()

def NDCG_at_K(groundTruth, r, k):
    assert len(r) == len(groundTruth)
    test_matrix = torch.zeros((len(r), k))
    for i, items in enumerate(groundTruth):
        length = min(len(items), k)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = torch.sum(max_r * 1. / torch.log2(torch.arange(2, k + 2)), axis=1)
    dcg = r * (1. / torch.log2(torch.arange(2, k + 2)))
    dcg = torch.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg / idcg
    ndcg[torch.isnan(ndcg)] = 0.
    #
    return torch.mean(ndcg).item()

def get_user_positive_items(edge_index):
    user_pos_items = {}
    for i in range(edge_index.shape[1]):
        user = edge_index[0][i].item()
        item = edge_index[1][i].item()
        if user not in user_pos_items:
            user_pos_items[user] = []
        user_pos_items[user].append(item)
        
    return user_pos_items

from torch_geometric.utils import negative_sampling

from tqdm import tqdm

def get_metrics_with_negative_sampling(
    model, edge_index, sparse_edge_index, exclude_edge_index, train_sparse_edge_index, k, num_neg_samples=100
):
    """
    Evaluate the model using explicit negative sampling.
    
    Parameters:
        - model: The trained model.
        - edge_index: Test edge index (user-item interactions).
        - sparse_edge_index: Sparse test edge matrix.
        - exclude_edge_index: Edges to exclude (train + validation).
        - train_sparse_edge_index: Sparse adjacency matrix for training.
        - k: Number of top items to evaluate (Recall@K, etc.).
        - num_neg_samples: Number of negative samples per user.
    Returns:
        - recall, precision, ndcg
    """
    model.eval()
    # Get user and item embeddings
    user_embedding, _, item_embedding, _, _ = model.forward(train_sparse_edge_index)
    user_embedding = user_embedding.cpu().detach().numpy()
    item_embedding = item_embedding.cpu().detach().numpy()
    
    rating = torch.tensor(np.matmul(user_embedding, item_embedding.T))

    # Mask out all positive interactions from train and test data
    user_pos_items = get_user_positive_items(exclude_edge_index)
    exclude_users = []
    exclude_items = []
    for user, items in user_pos_items.items():
        exclude_users.extend([user] * len(items))
        exclude_items.extend(items)
    rating[exclude_users, exclude_items] = float('-inf')

    # Generate negative samples
    num_users = rating.shape[0]
    num_items = rating.shape[1]
    all_items = torch.arange(num_items)
    neg_samples = {}
    for user in range(num_users):
        pos_items = set(user_pos_items.get(user, []))
        neg_items = list(set(all_items.tolist()) - pos_items)
        neg_samples[user] = torch.tensor(neg_items, dtype=torch.long)[
            torch.randperm(len(neg_items))[:num_neg_samples]
        ]

    # Evaluate on positive test items + negative samples
    users = edge_index[0].unique()
    test_user_pos_items = get_user_positive_items(edge_index)
    test_user_pos_items_list = [test_user_pos_items[user.item()] for user in users]
    
    recall = 0.0
    precision = 0.0
    ndcg = 0.0

    for user in users:
        user = user.item()
        # Combine test positives and sampled negatives
        test_items = set(test_user_pos_items.get(user, []))
        sampled_items = list(test_items.union(neg_samples[user].tolist()))
        
        # Get top-K recommendations
        user_ratings = rating[user, sampled_items]
        _, top_K_items = torch.topk(user_ratings, k=k)
        top_K_items = [sampled_items[i] for i in top_K_items]

        # Create relevance vector
        ground_truth_items = set(test_user_pos_items.get(user, []))
        relevance = torch.tensor(
            [1.0 if item in ground_truth_items else 0.0 for item in top_K_items]
        )

        # Calculate metrics
        recall += torch.sum(relevance) / len(ground_truth_items)
        precision += torch.sum(relevance) / k

        # Calculate NDCG
        gains = relevance / torch.log2(torch.arange(2, k + 2).float())
        dcg = torch.sum(gains)
        ideal_gains = torch.zeros_like(relevance)
        ideal_gains[: len(ground_truth_items)] = 1.0
        idcg = torch.sum(ideal_gains / torch.log2(torch.arange(2, k + 2).float()))
        ndcg += dcg / idcg

    num_users = len(users)
    recall /= num_users
    precision /= num_users
    ndcg /= num_users

    return recall, precision, ndcg

def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_):
    reg_loss = lambda_ * (users_emb_0.norm(2).pow(2) +
                          pos_items_emb_0.norm(2).pow(2) +
                          neg_items_emb_0.norm(2).pow(2)) # L2 loss

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1)
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1)
    
    loss = -F.logsigmoid(pos_scores - neg_scores).sum() + reg_loss
    
    return loss

import torch
import torch.nn.functional as F

def custom_negative_sampling(edge_index, num_nodes, num_neg_samples):
    """
    edge_index: Sparse edge indices of the graph (positive edges)
    num_nodes: Total number of nodes in the graph
    num_neg_samples: Number of negative samples per positive sample
    num_users: Number of user nodes in the graph
    num_courses: Number of course nodes in the graph
    """
    # Extract rows and columns from edge_index
    row, col = edge_index
    row_device = row.device  # Ensure device compatibility

    # Sample negative nodes outside the user-course range
    valid_neg_nodes = torch.cat([
        torch.arange(num_users, device=row_device),
        torch.arange(num_users + num_courses, num_nodes, device=row_device)  
    ])
    
    neg_col = valid_neg_nodes[torch.randint(0, valid_neg_nodes.size(0), (row.size(0) * num_neg_samples,))]

    # Ensure negative samples are unique and do not overlap with positive edges
    mask = torch.isin(neg_col, col)  # Overlap with existing edges
    while mask.any():
        neg_col[mask] = valid_neg_nodes[
            torch.randint(0, valid_neg_nodes.size(0), (mask.sum(),))
        ]
        mask = torch.isin(neg_col, col)  # Recheck for overlaps

    # Construct negative edge index
    neg_edge_index = torch.stack([row.repeat_interleave(num_neg_samples), neg_col]).t()

    return neg_edge_index


In [9]:
import torch
torch.autograd.set_detect_anomaly(True)
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
epochs = 20  # Number of epochs
check_step = 4  # Evaluate every `check_step` epochs
batch_size = 2048  # Batch size for training
lambda_ = 0.0001  # Regularization parameter

# Initialize the model, optimizer, and other components
model = HeteroGATModel(in_dim = 32, hidden_dim = 64, out_dim = 32, num_layers = 2, heads=1, dropout=0.3)
model.to(device)  # Move the model to the device (GPU or CPU)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-5)
kg_optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-5)

for epoch in range(1, epochs + 1):
    model.train()
    trn_loader = DataLoader(train_edge_index.T, batch_size=batch_size, shuffle=True)
    trn_loss = 0

    # Wrap the DataLoader in tqdm to track batches
    batch_bar = tqdm(trn_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
    for batch_idx, batch_pos_edges in enumerate(batch_bar):
        batch_pos_edges = batch_pos_edges.T
        batch_pos_edges = batch_pos_edges.to(device)  # Move batch to device

        # Forward pass with multiple adjacency matrices
        user_emb_final, user_emb_initial, course_emb_final, course_emb_initial, other_emb_final = model.forward(
            concatenated_sparse_edge_index.to(device)
        )

        # Generate negative samples for the batch
        batch_neg_edges = negative_sampling(
            train_edge_index.to(device),  # Ensure train_edge_index is on the same device
            num_nodes=[num_users, num_courses],
            num_neg_samples=batch_pos_edges.shape[1],
        ).to(device)  # Ensure negative samples are on the same device

        # Extract indices for users, positive items, and negative items
        user_indices = batch_pos_edges[0].to(device)
        pos_item_indices = batch_pos_edges[1].to(device)
        neg_item_indices = batch_neg_edges[1].to(device)

        # Embed users and items based on the indices
        users_emb_final = user_emb_final[user_indices]
        users_emb_initial = user_emb_initial[user_indices]
        pos_items_emb_final = course_emb_final[pos_item_indices]
        neg_items_emb_final = course_emb_final[neg_item_indices]
        pos_items_emb_initial = course_emb_initial[pos_item_indices]
        neg_items_emb_initial = course_emb_initial[neg_item_indices]

        # Calculate BPR loss
        loss = bpr_loss(
            users_emb_final, 
            users_emb_initial, 
            pos_items_emb_final, 
            pos_items_emb_initial, 
            neg_items_emb_final, 
            neg_items_emb_initial, 
            lambda_
        )

        all_emb_final = torch.cat([user_emb_final, course_emb_final, other_emb_final], dim=0)
        
        row_indices = concatenated_sparse_edge_index.storage.row().to(device)
        col_indices = concatenated_sparse_edge_index.storage.col().to(device)
    
        mask = torch.isin(row_indices, batch_pos_edges[1])
    
        course_indices = row_indices[mask]
        other_indices = col_indices[mask]
    
        pos_edge_index = torch.stack([course_indices, other_indices], dim=0).to(device)
    
        # Negative sampling
        neg_edge_index= custom_negative_sampling(
            pos_edge_index, num_users + num_courses + num_others, 1
        )

        del row_indices
        del col_indices
    
        pos_course_emb = all_emb_final[course_indices].to(device)
        pos_other_emb = all_emb_final[other_indices].to(device)
    
        neg_row_indices = neg_edge_index[:, 0].to(device)
        neg_col_indices = neg_edge_index[:, 1].to(device)

        neg_course_emb = all_emb_final[neg_row_indices].to(device)
        neg_other_emb = all_emb_final[neg_col_indices].to(device)  # Negative samples
    
        # Positive and negative scores
        other_pos_scores = torch.sum(pos_course_emb * pos_other_emb, dim=1)
        other_neg_scores = torch.sum(neg_course_emb * neg_other_emb, dim=1)
        
        kg_loss = torch.mean(torch.clamp(1 - other_pos_scores, min=0)) + \
                  torch.mean(torch.clamp(other_neg_scores + 1, min=0))

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        kg_optimizer.zero_grad()
        kg_loss.backward()
        optimizer.step()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        trn_loss += loss.item() + kg_loss.item()

        # Update tqdm with current batch loss
        batch_bar.set_postfix(batch_loss=loss.item() + kg_loss.item())

    trn_loss = trn_loss / len(trn_loader)
    print(f"Epoch {epoch}/{epochs} - Training loss: {trn_loss:.6f}")

    # Evaluate and display metrics every `check_step` epochs
    if epoch != 0 and epoch % check_step == 0:
        model.eval()
        recall, precision, ndcg = get_metrics_with_negative_sampling(
            model, 
            test_edge_index.to(device),  
            test_sparse_edge_index.to(device),
            train_edge_index.to(device),  
            concatenated_sparse_edge_index.to(device),
            k=10
        )
        score = 0.75 * recall + 0.25 * ndcg

        print(f'[{epoch:03d}/{epochs}] | loss: {trn_loss:.6f} | recall@{10}: {recall:.6f} | '
              f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

Epoch 1/20 - Training loss: 416.945478


Epoch 2/20 - Training loss: 345.266812


Epoch 3/20 - Training loss: 318.292358


Epoch 4/20 - Training loss: 308.925775
[004/20] | loss: 308.925775 | recall@10: 0.688457 | precision@10: 0.068889 | ndcg@10: 0.438694 | score: 0.626016


Epoch 5/20 - Training loss: 302.039998


...

Epoch 16/20 - Training loss: 273.401425
[016/20] | loss: 273.401425 | recall@10: 0.711834 | precision@10: 0.071229 | ndcg@10: 0.462561 | score: 0.649515


Epoch 17/20 - Training loss: 271.669259


Epoch 18/20 - Training loss: 270.561712


Epoch 19/20 - Training loss: 269.489528


Epoch 20/20 - Training loss: 266.750974
[020/20] | loss: 266.750974 | recall@10: 0.712614 | precision@10: 0.071307 | ndcg@10: 0.465716 | score: 0.650889


In [10]:
model.eval()
recall, precision, ndcg = get_metrics_with_negative_sampling(
    model, 
    test_edge_index.to(device),
    test_sparse_edge_index.to(device),
    train_edge_index.to(device),
    concatenated_sparse_edge_index.to(device),
    k=10
)
score = 0.75 * recall + 0.25 * ndcg

print(f'| recall@{10}: {recall:.6f} | '
      f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

| recall@10: 0.712444 | precision@10: 0.071290 | ndcg@10: 0.466541 | score: 0.650968


In [11]:
model.eval()
recall, precision, ndcg = get_metrics_with_negative_sampling(
    model, 
    test_edge_index.to(device),
    test_sparse_edge_index.to(device),
    train_edge_index.to(device),
    concatenated_sparse_edge_index.to(device),
    k=5
)
score = 0.75 * recall + 0.25 * ndcg

print(f'| recall@{10}: {recall:.6f} | '
      f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

| recall@10: 0.567740 | precision@10: 0.113612 | ndcg@10: 0.419945 | score: 0.530791


In [12]:
model.eval()
recall, precision, ndcg = get_metrics_with_negative_sampling(
    model, 
    test_edge_index.to(device),
    test_sparse_edge_index.to(device),
    train_edge_index.to(device),
    concatenated_sparse_edge_index.to(device),
    k=1
)
score = 0.75 * recall + 0.25 * ndcg

print(f'| recall@{10}: {recall:.6f} | '
      f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

| recall@10: 0.258778 | precision@10: 0.258778 | ndcg@10: 0.258778 | score: 0.258778


In [13]:
data_path = '/kaggle/input/mooccubex/test_df.csv'
df = pd.read_csv(data_path)
df = df[df['test_label'] < num_courses]

test_user_col = torch.tensor(df['user'].values, dtype=torch.int64)
test_course_col = torch.tensor(df['test_label'].values, dtype=torch.int64)

test_edge_index = torch.stack([test_user_col, test_course_col], dim=0)

test_course_col_shifted = test_course_col + num_users
test_sparse_edge_index = SparseTensor(
    row=test_user_col,
    col=test_course_col_shifted,
    sparse_sizes=(num_users + num_courses, num_users + num_courses)
)

In [14]:
model.eval()
recall, precision, ndcg = get_metrics_with_negative_sampling(
    model, 
    test_edge_index.to(device),
    test_sparse_edge_index.to(device),
    train_edge_index.to(device),
    concatenated_sparse_edge_index.to(device),
    k=10
)
score = 0.75 * recall + 0.25 * ndcg

print(f'| recall@{10}: {recall:.6f} | '
      f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

| recall@10: 0.660064 | precision@10: 0.066047 | ndcg@10: 0.416637 | score: 0.599208


In [15]:
model.eval()
recall, precision, ndcg = get_metrics_with_negative_sampling(
    model, 
    test_edge_index.to(device),
    test_sparse_edge_index.to(device),
    train_edge_index.to(device),
    concatenated_sparse_edge_index.to(device),
    k=5
)
score = 0.75 * recall + 0.25 * ndcg

print(f'| recall@{10}: {recall:.6f} | '
      f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

| recall@10: 0.508328 | precision@10: 0.101718 | ndcg@10: 0.367275 | score: 0.473065


In [16]:
model.eval()
recall, precision, ndcg = get_metrics_with_negative_sampling(
    model, 
    test_edge_index.to(device),
    test_sparse_edge_index.to(device),
    train_edge_index.to(device),
    concatenated_sparse_edge_index.to(device),
    k=1
)
score = 0.75 * recall + 0.25 * ndcg

print(f'| recall@{10}: {recall:.6f} | '
      f'precision@{10}: {precision:.6f} | ndcg@{10}: {ndcg:.6f} | score: {score:.6f}')

| recall@10: 0.215470 | precision@10: 0.215470 | ndcg@10: 0.215470 | score: 0.215470


In [17]:
checkpoint = {'model': HeteroGATModel(in_dim = 32, hidden_dim = 64, out_dim = 32, num_layers = 2, heads=1, dropout=0.3),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict(),
              'kg_optimizer': kg_optimizer.state_dict()
             }

torch.save(checkpoint, 'checkpoint.pth')

In [26]:
!pip install googletrans==4.0.0-rc1
from googletrans import Translator

translator = Translator()

def translate_column(df, column_name):
    """
    Translates the text in a specified column of the DataFrame to the desired language.

    Args:
        df (pd.DataFrame): The DataFrame containing the column to translate.
        column_name (str): The name of the column to translate.

    Returns:
        pd.DataFrame: DataFrame with the translated column.
    """
    df[column_name] = df[column_name].apply(
        lambda text: translator.translate(text = text, src='zh-CN', dest='vi').text if pd.notna(text) else text
    )
    
    return df

def recommend_courses_for_user(
    model, user_id, train_sparse_edge_index, exclude_edge_index, top_k=10
):
    """
    Recommend courses for a given user ID.
    
    Parameters:
        - model: The trained recommendation model.
        - user_id: The user ID to generate recommendations for.
        - train_sparse_edge_index: Sparse adjacency matrix for training.
        - exclude_edge_index: Edges to exclude (train + validation interactions).
        - top_k: Number of top items to recommend (default: 10).
    
    Returns:
        - A list of top_k recommended course IDs.
    """
    model.eval()

    # Get user and item embeddings
    user_embedding, _, item_embedding, _, _ = model.forward(train_sparse_edge_index)
    user_embedding = user_embedding.cpu().detach().numpy()
    item_embedding = item_embedding.cpu().detach().numpy()

    # Compute scores for all items for the given user
    user_vector = user_embedding[user_id]
    scores = np.dot(user_vector, item_embedding.T)

    # Mask out already interacted items
    user_pos_items = get_user_positive_items(exclude_edge_index)
    exclude_items = set(user_pos_items.get(user_id, []))
    scores[list(exclude_items)] = float('-inf')  # Set scores for interacted items to -inf

    # Get the top-K items
    top_k_items = np.argsort(-scores)[:top_k]  # Sort in descending order and pick top K

    return top_k_items


In [27]:
user_id = 123
recommended_courses = recommend_courses_for_user(
    model,
    user_id=user_id,
    train_sparse_edge_index=concatenated_sparse_edge_index.to(device),
    exclude_edge_index=torch.cat([train_edge_index, test_edge_index], dim = -1).to(device),
    top_k=10
)

print(f"Recommended courses for user {user_id}: {recommended_courses}")


Recommended courses for user 123: [1956 2342 2699 1966  497  422 2293 2055 2330  697]


In [45]:
course_map_path = "/kaggle/input/mooccubex/course_map.csv"
course_prepared_path = "/kaggle/input/mooccubex/course_prepared.json"

# Read data
course_map = pd.read_csv(course_map_path, names=['course', 'map'])
course_mapping = dict(zip(course_map["course"], course_map["map"]))
course_prepared = pd.read_json(course_prepared_path, lines = True)
course_prepared["id"] = course_prepared["id"].map(course_mapping)
course_prepared = course_prepared.dropna(subset=["id"])

predicted_course = course_prepared[course_prepared['id'].isin(recommended_courses)]
translate_column(predicted_course, 'name')
translate_column(predicted_course, 'prerequisites')
translate_column(predicted_course, 'about')

predicted_course

/tmp/ipykernel_23/2266137368.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(


,id,name,prerequisites,about,resource,field,schools,teachers
760,497.0,Cấu trúc xây dựng nhà ở,Vẽ và hiểu kỹ thuật xây dựng,Cấu trúc xây dựng của ngôi nhà là một khóa học...,"[{'titles': ['第一章 绪论', '1.1 建筑的基本要素', '1.1.1 建...",None,[S_49],"[T_2903, T_2250, T_4461, T_4459, T_2251, T_290..."
1171,422.0,Nhà thiết kế tinh tế,None,Nhà là một container cho cuộc sống.Làm thế nào...,"[{'titles': ['导论', '绪论', '绪论'], 'resource_id':...",[建筑学],[S_1],"[T_640, T_640, T_640, T_640, T_640]"
1617,2055.0,Công nghệ xây dựng,Làm chủ kiến ​​thức liên quan về hình ảnh kiến...,Bạn muốn tìm hiểu các phương pháp xây dựng và ...,"[{'titles': ['项目一：土方工程施工', '任务1.1土的工程分类', '任务1...",None,[S_513],"[T_5352, T_5353, T_5354, T_5355, T_5356, T_535..."
2222,2342.0,Công nghệ xây dựng,Các khóa học công nghệ xây dựng là một khóa họ...,Công nghệ xây dựng là một khóa học cốt lõi chu...,"[{'titles': ['项目一 土方工程施工', '1.2 场地平整', '场地平整']...",None,[S_669],"[T_6193, T_6194, T_6195, T_6196, T_6197, T_619..."
2278,1966.0,Thiết kế mẫu quần áo,None,Khóa học là khóa học cốt lõi của trường và đan...,"[{'titles': ['第一章 概论', '1.1 概论', '1.1视频 概论上'...",None,[S_712],"[T_8079, T_8081, T_8083, T_8085, T_8086, T_807..."
2281,2330.0,Thiết kế trang phục nam,None,"""Trong giảng dạy đại học truyền thống, kết hợp...","[{'titles': ['第1章 概述', '1.1 男装设计的概念与分类', '视频录课...",None,[S_712],"[T_9118, T_9121, T_9127, T_9128, T_9118, T_912..."
2537,1956.0,Thiết kế phác thảo,"""Phác thảo cơ bản"", Quan điểm","Phim và truyền hình, quần áo, quảng cáo, trong...","[{'titles': ['第一章 设计素描概述', '1.1 设计素描的基本特征', '1...",None,[S_1087],"[T_10631, T_10636, T_10632, T_10633, T_10634, ..."
2765,2293.0,21 Lớp học tiếng Anh sau đại học,None,21 Lớp học tiếng Anh sau đại học,"[{'titles': ['21考研英语小白课堂', None, '21考研英语备考高分秘籍...",None,[S_63],[T_11805]
2821,697.0,Sơ đồ kỹ thuật,None,Các khóa học vẽ kỹ thuật không chỉ dạy bạn xem...,"[{'titles': ['第一讲：第1章 绪论', '1.1 工程制图的应用背景', '1...",[机械工程],[S_1],"[T_508, T_509, T_510, T_511, T_512, T_513, T_5..."
3215,2699.0,Kiến trúc nhà ở,None,Cuộc sống của mọi người có liên quan chặt chẽ ...,"[{'titles': ['第一章 建筑设计概论', '1.1 认识建筑', '1.1 认识...",None,[S_1306],"[T_13604, T_13604]"
